# Set up Account Consent APIs
Running this script will create the APIs that we will be monitoring for accounts consents. They do not
require user authentication. 

The API setups are defined with tags that allow us to easily provide OBIE metric reporting.

## Requirements
1. Create an APImetrics Project
1. Get an API key with EDITOR permissions for the project, enter it below
1. Get the maTLS transport certificate and key in PEM format, enter paths below
1. Run this workbook to create all APIs
1. Create an environment variable "financial_id" - TBD - add this here
1. Create a Worfklow for the Account Consent API calls - TDB - add this here


In [ ]:
# Enter intended project's APImetrics API key here:
API_KEY = ""

# Enter the bank's full path, including https and without trailing /
BANK_URL = 'https://example.com/open-banking/3.1/aisp'

SSL_KEY_PATH = "" #"./local/TPP_OB_Transport.key"
SSL_CERT_PATH = "" #"./local/TTP_OB_Transport.pem"

OAUTH_METHOD = "private_key_jwt"
# if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
# # CLIENT_ID = '...'
# # CLIENT_SECRET = '...'

# The OBIE version ID we're working against
VERSION = "3.1"

from urllib.parse import urlparse

url_info = urlparse(BANK_URL)
DOMAIN, _, _ = url_info.netloc.partition(':') # removes the port 

In [ ]:
# Helper functions
import requests
import json

class APImetricsAPI:

    BASE_URL = "https://client.apimetrics.io/api/2"
    GET_HEADERS = {"Accept": "application/json"}
    POST_HEADERS = {"Content-Type": "application/json", **GET_HEADERS}

    def __init__(self, api_key):
        self.api_key = api_key
        self._auths = None
        self._tokens = None
        self._calls = None
        self._workflows = None
        self._auths_by_tag = None
        self._tokens_by_auth = None
        self._calls_by_tag = None
        self._workflows_by_tag = None
        
    @property
    def auths(self):
        if self._auths is None:
            self._auths = { obj['id']: obj for obj in self.get_auths().get('results', []) }
            print(f'Found {len(self._auths)} Auths')
        return self._auths
    
    @property
    def auths_by_tag(self):
        if self._auths_by_tag is None:
            self._auths_by_tag = {}
            for auth in self.auths.values():
                for tag in auth['meta']['tags']:
                    if ':' in tag:
                        self._auths_by_tag[tag] = auth['id']
            print(f'Auth tags: {", ".join(self._auths_by_tag.keys())}')
        return self._auths_by_tag
    
    @property
    def tokens(self):
        if self._tokens is None:
            self._tokens = { obj['id']: obj for obj in self.get_tokens().get('results', []) }
            print(f'Found {len(self._tokens)} Tokens')
        return self._tokens
    
    @property
    def tokens_by_auth(self):
        if self._tokens_by_auth is None:
            self._tokens_by_auth = {}
            for token in self.tokens.values():
                auth_id = token['meta']['auth_id']
                self._tokens_by_auth[auth_id] = token['id']
            print(f'Tokens for auths: {", ".join(self.auths[auth_id]["meta"]["name"] for auth in self._tokens_by_auth.keys())}')
        return self._tokens_by_auth
    
    @property
    def calls(self):
        if self._calls is None:
            self._calls = { obj['id']: obj for obj in CLIENT.get_calls().get('results', []) }
            print(f'Found {len(self._calls)} Calls')
        return self._calls

    @property   
    def calls_by_tag(self):
        if self._calls_by_tag is None:
            self._calls_by_tag = {}
            for call in self.calls.values():
                for tag in call['meta']['tags']:
                    if ':' in tag:
                        self._calls_by_tag[tag] = call['id']
            print(f'Call tags: {", ".join(self._calls_by_tag.keys())}')
        return self._calls_by_tag
    
    @property
    def workflows(self):
        if self._workflows is None:
            self._workflows = { obj['id']: obj for obj in CLIENT.get_workflows().get('results', []) }
            print(f'Found {len(self._workflows)} Workflows')
        return self._workflows
    
    @property   
    def workflows_by_tag(self):
        if self._workflows_by_tag is None:
            self._workflows_by_tag = {}
            for workflow in self.workflows.values():
                for tag in workflow['meta']['tags']:
                    if ':' in tag:
                        self._workflows_by_tag[tag] = workflow['id']
            print(f'Call tags: {", ".join(self._workflows_by_tag.keys())}')
        return self._workflows_by_tag
        
    def headers(self, is_post=False):
        other_headers = self.POST_HEADERS if is_post else self.GET_HEADERS
        return {"Authorization": "Bearer {}".format(self.api_key), **other_headers}

    def get(self, path, params):
        resp = requests.get(
            "{}/{path}".format(self.BASE_URL, path=path),
            headers=self.headers(),
            params=params,
        )
        try:
            resp.raise_for_status()
        except Exception as ex:
            print(resp.content)
            raise ex
        return resp.json()

    def post(self, path, setup):
        resp = requests.post(
            "{}/{path}".format(self.BASE_URL, path=path),
            json=setup,
            headers=self.headers(True),
        )
        try:
            resp.raise_for_status()
        except Exception as ex:
            print(resp.content)
            raise ex
        return resp.json()
    
    def delete(self, path):
        resp = requests.delete(
            "{}/{path}".format(self.BASE_URL, path=path),
            headers=self.headers(),
        )
        try:
            resp.raise_for_status()
        except Exception as ex:
            print(resp.content)
            raise ex
        return True
    
    def get_all(self, path):
        cursor = None
        more = True
        results = []
        while more:
            resp = self.get(path, {'cursor': cursor})
            results.extend(resp.get('results', []))
            more = resp['meta']['more']
            cursor = resp['meta']['next_cursor']
        resp['results'] = results
        return resp

    def get_auths(self):
        return self.get_all("auth/")

    def get_calls(self):
        return self.get_all("calls/")

    def get_schedules(self):
        return self.get_all("schedules/")

    def get_tokens(self):
        return self.get_all("tokens/")

    def get_token(self, id_str):
        return self.get("tokens/{}".format(id_str))

    def get_tokens_for_auth(self, auth_id):
        return self.get("tokens/auth/{}/".format(auth_id))

    def get_workflows(self):
        return self.get_all("workflows")

    def create_auth(self, setup):
        self._auths = None
        self._auths_by_tag = None
        return self.post("auth/", setup)

    def create_call(self, setup):
        self._calls = None
        self._calls_by_tag = None
        return self.post("calls/", setup)

    def create_token(self, setup):
        self._tokens = None
        self._tokens_by_auth = None
        return self.post("tokens/", setup)

    def create_workflow(self, setup):
        self._workflows = None
        self._workflows_by_tag = None
        return self.post("workflows/", setup)

    def update_auth(self, id_str, setup):
        return self.post("auth/{}/".format(id_str), setup)

    def update_call(self, id_str, setup):
        return self.post("calls/{}/".format(id_str), setup)

    def update_token(self, id_str, setup):
        return self.post("tokens/{}/".format(id_str), setup)

    def update_workflow(self, id_str, setup):
        return self.post("workflows/{}/".format(id_str), setup)

    def delete_auth(self, id_str):
        self._auths = None
        self._auths_by_tag = None
        return self.delete("auth/{}/".format(id_str))

    def delete_call(self, id_str):
        self._calls = None
        self._calls_by_tag = None
        return self.delete("calls/{}/".format(id_str))

    def delete_token(self, id_str):
        self._tokens = None
        self._tokens_by_auth = None
        return self.delete("tokens/{}/".format(id_str))

    def delete_workflow(self, id_str):
        self._workflows = None
        self._workflows_by_tag = None
        return self.delete("workflows/{}/".format(id_str))
    
    def get_env_variable(self, env, key):
        return self.get("environment/{env}/{key}".format(env=env, key=key))

    def set_env_variable(self, env, key, val):
        return self.post(
            "environment/{env}/{key}".format(env=env, key=key), {"value": val}
        )

    def run_call(self, id_str, config):
        return self.post("calls/{}/run".format(id_str), config)
    
# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

# [CLIENT.delete_token(o['id']) for o in CLIENT.tokens.values()]
# [CLIENT.delete_auth(o['id']) for o in CLIENT.auths.values()]
# [CLIENT.delete_call(o['id']) for o in CLIENT.calls.values()]
# [CLIENT.delete_workflow(o['id']) for o in CLIENT.workflows.values()]

## Create Auth Setting and Token for Bank

In [ ]:
# First create Auth Setting
tag = 'auth:bank_matls'
if tag not in CLIENT.auths_by_tag:

    ssl_key = None
    ssl_cert = None

    if SSL_KEY_PATH:
        with open(SSL_KEY_PATH) as stream:
            ssl_key = stream.read()

    if SSL_CERT_PATH:
        with open(SSL_CERT_PATH) as stream:
            ssl_cert = stream.read()
    
    setup = {
        "access": {
            "keys": False,
            "org_keys": False,
            "org_settings": True,
            "settings": False,
        },
        "keys": {},
        "meta": {
            "domain": DOMAIN,
            "documentation": {"keys": "", "docs": "", "apps": "", "provider": ""},
            "name": "Transport MATLS",
            "tags": [tag],
            "description": "Mutual Authenticated TLS for calls to bank APIs",
        },
        "settings": {
            "auth_type": "MANUAL",
            "ssl_key": ssl_key,
            "ssl_cert": ssl_cert,
        },
    }
    if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
        setup["keys"]["client_id"] = CLIENT_ID
        setup["keys"]["client_secret"] = CLIENT_SECRET
    
    auth = CLIENT.create_auth(setup)
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")

## Create APIs

In [ ]:
# Account Access Consent Create (minimal)
tag = f"banks:{VERSION}:account-access-consents:create_min"
if tag not in CLIENT.calls_by_tag:
    body = {"Data": {"Permissions": ["ReadAccountsBasic"]}, "Risk": {}}
    body_str = json.dumps(body, indent=2)
    setup = {
        "meta": {
            "description": None,
            "tags": ["api_type:create", "sector:financial", "ob_id:1", f"ob_v:{VERSION}"]
            + [tag],
            "name": f"v{VERSION}: Account-Access-Consents: Create (minimal)",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [],
            "url": "{}/account-access-consents".format(BANK_URL),
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": [
                {"value": "application/json", "key": "Accept"},
                {"value": "application/json", "key": "Content-Type"},
                {"value": "Bearer %%ACCESS_TOKEN%%", "key": "Authorization"},
                {"value": "{{financial_id}}", "key": "x-fapi-financial-id"},
                {"value": "%%GUID%%", "key": "x-fapi-interaction-id"},
                {
                    "value": "APIMetrics-%%TEST_RUN_RESULT_ID%%",
                    "key": "x-idempotency-key",
                },
            ],
            "token_id": None,
            "method": "POST",
        },
    }
    data = CLIENT.create_call(setup)
    print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
# Account Access Consent Create (maximal)
tag = f"banks:{VERSION}:account-access-consents:create_max"
if tag not in CLIENT.calls_by_tag:
    body = {
        "Data": {
            "Permissions": [
                "ReadAccountsDetail",
                "ReadBalances",
                "ReadBeneficiariesDetail",
                "ReadDirectDebits",
                "ReadOffers",
                "ReadParty",
                "ReadPartyPSU",
                "ReadProducts",
                "ReadStandingOrdersDetail",
                "ReadScheduledPaymentsDetail",
                "ReadTransactionsCredits",
                "ReadTransactionsDebits",
                "ReadTransactionsDetail",
                "ReadStatementsDetail",
            ]
        },
        "Risk": {},
    }
    body_str = json.dumps(body, indent=2)
    setup = {
        "meta": {
            "description": None,
            "tags": ["api_type:create", "sector:financial", "ob_id:1", f"ob_v:{VERSION}"]
            + [tag],
            "name": f"v{VERSION}: Account-Access-Consents: Create (maximal)",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [],
            "url": "{}/account-access-consents".format(BANK_URL),
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": [
                {"value": "application/json", "key": "Accept"},
                {"value": "application/json", "key": "Content-Type"},
                {"value": "Bearer %%ACCESS_TOKEN%%", "key": "Authorization"},
                {"value": "{{financial_id}}", "key": "x-fapi-financial-id"},
                {"value": "%%GUID%%", "key": "x-fapi-interaction-id"},
                {
                    "value": "APIMetrics-%%TEST_RUN_RESULT_ID%%",
                    "key": "x-idempotency-key",
                },
            ],
            "token_id": None,
            "method": "POST",
        },
    }
    data = CLIENT.create_call(setup)
    print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
# Account Access Consent Read
tag = f"banks:{VERSION}:account-access-consents:read"
if tag not in CLIENT.calls_by_tag:
    setup = {
        "meta": {
            "description": None,
            "tags": ["api_type:read", "sector:financial", "ob_id:2", f"ob_v:{VERSION}"]
            + [tag],
            "name": "v3.1: Account-Access-Consents: Get",
            "workspace": "global",
        },
        "request": {
            "body": None,
            "parameters": [],
            "url": "{}/account-access-consents/__CONSENT_ID__".format(BANK_URL),
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": [
                {"value": "application/json", "key": "Accept"},
                {"value": "application/json", "key": "Content-Type"},
                {"value": "Bearer %%ACCESS_TOKEN%%", "key": "Authorization"},
                {"value": "{{financial_id}}", "key": "x-fapi-financial-id"},
                {"value": "%%GUID%%", "key": "x-fapi-interaction-id"},
                {
                    "value": "APIMetrics-%%TEST_RUN_RESULT_ID%%",
                    "key": "x-idempotency-key",
                },
            ],
            "token_id": None,
            "method": "GET",
        },
    }
    data = CLIENT.create_call(setup)
    print(f"Created API {data['meta']['name']} with id {data['id']}")

In [ ]:
# Account Access Consent Delete
tag = f"banks:{VERSION}:account-access-consents:delete"
if tag not in CLIENT.calls_by_tag:
    setup = {
        "meta": {
            "description": None,
            "tags": ["api_type:delete", "sector:financial", "ob_id:3", "ob_v:3.1", tag],
            "name": "v3.1: Account-Access-Consents: Delete",
            "workspace": "global",
        },
        "request": {
            "body": None,
            "parameters": [],
            "url": "{}/account-access-consents/__CONSENT_ID__".format(BANK_URL),
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": [
                {"value": "Bearer %%ACCESS_TOKEN%%", "key": "Authorization"},
                {"value": "{{financial_id}}", "key": "x-fapi-financial-id"},
                {"value": "%%GUID%%", "key": "x-fapi-interaction-id"},
                {
                    "value": "APIMetrics-%%TEST_RUN_RESULT_ID%%",
                    "key": "x-idempotency-key",
                },
            ],
            "token_id": None,
            "method": "DELETE",
        },
    }
    data = CLIENT.create_call(setup)
    print(f"Created API {data['meta']['name']} with id {data['id']}")